# Turbofan Tutorial
`pyturbo`  library is provided by twiinIT to assembly a simple turbofan system.

The library is made of components: 

- `compressor` : fluid out is computed from fluid in and power provided by shaft with constant efficiency. 
- `combustor` : combustion is made considering constant FHV.
- `turbine` : power is extracted from fluid in considering a given expansion ratio and constant efficiency. 
- `inlet` and `nozzle` are computing `drag` and `thrust` from fluid conditions, ambiant pressure and throat section. 
- `nacelle`: envelop over the engine.
- `ogv`, `intermediate_casing` and `trf` and structures with aero channels. 

They are numerical components:

- `fluid_spitter` is used to split the flow into primary and secondary flow
- `shaft_spitter` is used to split the shaft power into booster and fan compressor

Aero 0D and simplified geometry are considered. 

A turbofan system is generated.

In [ ]:
from pyturbo.systems.turbofan import TurbofanWithAtm
sys = TurbofanWithAtm("sys")
tf = sys.tf
atm = sys.atm

In [ ]:
sys

In [ ]:
# geometrical view
sys.run_once()

tf.jupyter_view(options={
    "fan_module": dict(opacity=0.7, face_color="#92B4EC"),
    "fan_module.spinner": dict(face_color="#E1E5EA", opacity=1.),
    "fan_duct": dict(opacity=0.7),
    "core_cowl": dict(opacity=0.7),
    "nacelle": dict(face_color="#E1E5EA", opacity=0.6),
    "inlet": dict(opacity=1.),
    "gas_generator": dict(face_color="red", opacity=0.9),
    "turbine": dict(face_color="#92B4EC", opacity=0.9),
    "trf": dict(opacity=0.6),
})

# Simulation

The turbofan system has a couple of equations/unknowns to solve. We use `cosapp` non-linear solver for this purpose.

In [ ]:
from cosapp.drivers import NonLinearSolver

In [ ]:
from cosapp.utils import set_log, LogLevel

set_log(level=LogLevel.INFO)

## Direct mode
`thrust` is computed from `fuel_W`.

In [ ]:
# off-design solver
run = sys.add_driver(NonLinearSolver('run', max_iter=50, factor=0.8, history=False))

In [ ]:
# environment conditions
atm.altitude = 0.
atm.mach = 0.
atm.dtamb = 15.

# use case
tf.fuel_W = .5

sys.run_drivers()

print('mach =', atm.mach)
print('pamb =', tf.pamb, 'Pa')
print('thrust =', round(tf.thrust * 0.224809/1e3, 1), 'klbf')
print('N1 =', round(tf.N1), "rpm")
print('N2 =', round(tf.N2), "rpm")
print('bpr =', round(tf.bpr, 1))
print('opr =', round(tf.opr, 1))
print('T41 =', round(tf.core.turbine.fl_in.Tt), 'K')
print('sfc =', round(tf.sfc, 3), 'kg/(h*kN)')

In [ ]:
run.problem

## control mode
`fuel_W` is computed to match functional request, here the fan rotational speed `N1` value.

In [ ]:
# control solver
sys.drivers.clear()
run = sys.add_driver(NonLinearSolver('run'))
run.runner.add_unknown('tf.fuel_W')
run.runner.add_target('tf.N1')

In [ ]:
# environment conditions
atm.altitude = 0.
atm.mach = 0.
atm.dtamb = 15.

# use case
tf.N1 = 5000.

sys.run_drivers()

print('mach =', atm.mach)
print('pamb =', atm.pamb, 'Pa')
print('thrust =', round(tf.thrust * 0.224809/1e3, 1), 'klbf')
print('N1 =', round(tf.N1), "rpm")
print('N2 =', round(tf.N2), "rpm")
print('bpr =', round(tf.bpr, 1))
print('opr =', round(tf.opr, 1))
print('T41 =', round(tf.core.turbine.fl_in.Tt), 'K')
print('sfc =', round(tf.sfc, 3), 'kg/(h*kN)')

In [ ]:
run.problem

## design mode

Turbofan design characteristics are related to components and physical properties.

### Using design methods

In [ ]:
# design solver
sys.drivers.clear()
design = sys.add_driver(NonLinearSolver('design', max_iter=50, factor=0.8))
design.runner.add_unknown('tf.fuel_W')
design.runner.add_target('tf.thrust')

# engine
tf.thrust = 120e3
tf.bpr = 5.
tf.pr_nozzle = 1.1

# inlet
tf.inlet.aero.mach = 0.5

# fan module
tf.fan_module.fan.aero.pcnr = 0.95
tf.fan_module.fan.aero.utip = 420.

# booster
tf.fan_module.booster.aero.phi = 0.45
tf.fan_module.booster.aero.psi = 0.35
tf.fan_module.booster.aero.spec_flow = 180.
tf.fan_module.booster.aero.pcnr = 95.

# lpt
tf.turbine.aero.Ncqdes = 100.
tf.turbine.aero.psi = 1.25

# hpc
tf.core.compressor.aero.pr = 11.
tf.core.compressor.aero.utip = 420.
tf.core.compressor.aero.phi = 0.5
tf.core.compressor.aero.pcnr = 95.

# hpt
tf.core.turbine.aero.psi = 1.2
tf.core.turbine.aero.Ncqdes = 100.

# combustor
tf.core.combustor.aero.Tcomb = 1700.

# design method (using targets)
design.runner.design.extend(tf.design_methods['scaling'])

In [ ]:
# environment conditions
atm.altitude = 0.
atm.mach = 0.
atm.dtamb = 15.

sys.run_drivers()

print('mach =', atm.mach)
print('pamb =', atm.pamb, 'Pa')
print('W =', round(tf.fl_in.W), 'Kg/s')
print('thrust =', round(tf.thrust * 0.224809/1e3, 1), 'klbf')
print('N1 =', round(tf.N1), "rpm")
print('N2 =', round(tf.N2), "rpm")
print('bpr =', round(tf.bpr, 1))
print('opr =', round(tf.opr, 1))
print('T41 =', round(tf.core.turbine.fl_in.Tt), 'K')
print('sfc =', round(tf.sfc, 3), 'kg/(h*kN)')
print('psi fan =', round(tf.fan_module.fan.aero.psi, 2))
print('psi booster =', round(tf.fan_module.booster.aero.psi, 2))
print('psi hpc =', round(tf.core.compressor.aero.psi, 2))

### Using raw equations/unknowns (detailed)

In [ ]:
# design mode solver
sys.drivers.clear()
design = sys.add_driver(NonLinearSolver('design', max_iter=50, factor=0.8))

# engine
tf.thrust = 120e3
tf.bpr = 5.
tf.pr_nozzle = 1.1

# inlet
tf.inlet.aero.mach = 0.5

# fan module
tf.fan_module.fan.aero.pcnr = 0.95
tf.fan_module.fan.aero.utip = 420.

# booster
tf.fan_module.booster.aero.phi = 0.45
tf.fan_module.booster.aero.psi = 0.35
tf.fan_module.booster.aero.spec_flow = 180.
tf.fan_module.booster.aero.pcnr = 95.

# lpt
tf.turbine.aero.Ncqdes = 100.
tf.turbine.aero.psi = 1.25

# hpc
tf.core.compressor.aero.pr = 11.
tf.core.compressor.aero.utip = 420.
tf.core.compressor.aero.phi = 0.5
tf.core.compressor.aero.pcnr = 95.

# hpt
tf.core.turbine.aero.psi = 1.2
tf.core.turbine.aero.Ncqdes = 100.

# combustor
tf.core.combustor.aero.Tcomb = 1700.

# engine
design.runner.add_unknown('tf.fuel_W')
design.runner.add_target('tf.thrust')
design.add_unknown('tf.fan_diameter')

# inlet
design.runner.add_target('tf.inlet.aero.mach')

# fan
design.add_unknown("tf.fan_module.fan.aero.xnd", max_rel_step=0.5)
design.add_unknown('tf.fan_module.fan.aero.phiP', lower_bound=0.1, upper_bound=1.5)

design.runner.add_target("tf.fan_module.fan.aero.pcnr")
design.runner.add_target('tf.fan_module.fan.aero.utip')
design.runner.add_target('tf.bpr')

# booster
design.add_unknown('tf.fan_module.geom.booster_radius_ratio')
design.add_unknown('tf.fan_module.booster.geom.blade_hub_to_tip_ratio', lower_bound=1e-5, upper_bound=1.)
design.add_unknown('tf.fan_module.booster.aero.phiP')
design.add_unknown("tf.fan_module.booster.aero.xnd", max_rel_step=0.5)

design.runner.add_target('tf.fan_module.booster.aero.phi')
design.runner.add_target('tf.fan_module.booster.aero.psi')
design.runner.add_target('tf.fan_module.booster.aero.spec_flow')
design.runner.add_target("tf.fan_module.booster.aero.pcnr")

# lpt
design.add_unknown('tf.geom.turbine_radius_ratio')
design.add_unknown("tf.turbine.geom.blade_height_ratio", lower_bound=0., upper_bound=1.)
design.add_unknown('tf.turbine.aero.Ncdes')

design.runner.add_target('tf.turbine.aero.psi')
design.runner.add_target('tf.turbine.aero.Ncqdes')

# hpc
design.add_unknown('tf.geom.core_inlet_radius_ratio', max_rel_step=0.8)
design.add_unknown("tf.core.compressor.aero.xnd", max_rel_step=0.5)
design.add_unknown("tf.core.compressor.aero.phiP")

design.runner.add_target("tf.core.compressor.aero.pcnr")
design.runner.add_target("tf.core.compressor.aero.phi")
design.runner.add_target("tf.core.compressor.aero.utip")
design.runner.add_target("tf.core.compressor.aero.pr")

# combustor
design.runner.add_target("tf.core.combustor.aero.Tcomb")

# hpt
design.add_unknown('tf.geom.core_exit_radius_ratio', max_rel_step=0.8)
design.add_unknown("tf.core.turbine.geom.blade_height_ratio", lower_bound=0., upper_bound=1.)
design.add_unknown("tf.core.turbine.aero.Ncdes")

design.runner.add_target("tf.core.turbine.aero.psi")
design.runner.add_target("tf.core.turbine.aero.Ncqdes")

# nozzle
design.add_unknown('tf.geom.pri_nozzle_area_ratio', lower_bound=0.05)
design.add_unknown('tf.geom.sec_nozzle_area_ratio', upper_bound=1.)

design.runner.add_target('tf.pr_nozzle')

In [ ]:
# environment conditions
atm.altitude = 0.
atm.mach = 0.
atm.dtamb = 15.

sys.run_drivers()

print('mach =', atm.mach)
print('pamb =', atm.pamb, 'Pa')
print('fan diameter =', round(tf.geom.fan_diameter / 0.0254, 1), 'in')
print('W =', round(tf.fl_in.W), 'Kg/s')
print('thrust =', round(tf.thrust * 0.224809/1e3, 1), 'klbf')
print('N1 =', round(tf.N1), "rpm")
print('N2 =', round(tf.N2), "rpm")
print('bpr =', round(tf.bpr, 1))
print('opr =', round(tf.opr, 1))
print('T41 =', round(tf.core.turbine.fl_in.Tt), 'K')
print('sfc =', round(tf.sfc, 3), 'kg/(h*kN)')
print('psi fan =', round(tf.fan_module.fan.aero.psi, 2))
print('psi booster =', round(tf.fan_module.booster.aero.psi, 2))
print('psi hpc =', round(tf.core.compressor.aero.psi, 2))

In [ ]:
tf.jupyter_view(options={
    "fan_module": dict(opacity=0.7, face_color="#92B4EC"),
    "fan_module.spinner": dict(face_color="#E1E5EA", opacity=1.),
    "fan_duct": dict(opacity=0.3),
    "core_cowl": dict(opacity=0.7),
    "nacelle": dict(face_color="#E1E5EA", opacity=0.6),
    "inlet": dict(opacity=1.),
    "gas_generator": dict(face_color="red", opacity=0.9),
    "tcf": dict(face_color="#92B4EC", opacity=0.9),
    "turbine": dict(face_color="#92B4EC", opacity=0.9),
    "trf": dict(opacity=0.6),
})

### off-design computation after design

Fuel consumption for a given altitude/mach/dtamb and thrust. 

In [ ]:
# off-design mode
sys.drivers.clear()
run = sys.add_driver(NonLinearSolver('run'))

In [ ]:
# environment conditions
atm.altitude = 10000.
atm.mach = 0.8
atm.dtamb = 0.

# requirement
tf.thrust = 10e3 / 0.224809
# tf.N1 = 5000.

run.runner.add_unknown('tf.fuel_W')
run.runner.add_target('tf.thrust')
# run.runner.add_target('tf.N1')

sys.run_drivers()

print('fuel flow =', round(tf.fuel_W, 1), 'kg/s')
print('N1 =', round(tf.N1), "rpm")
print('sfc ', round(tf.sfc, 3), 'kg/(h*kN)')